In [29]:
import numpy as np
from vispy import app, scene
from vispy.geometry import Rect
# from funcs import init_boids
# from funcs import directions
app.use_app('pyqt5')

<Vispy app, wrapping the PyQt5 GUI toolkit>

In [30]:
w, h = 640, 480
N = 500
dt = 0.1
asp = w/h
perseption = 1/20
vrange = (0, 0.1)

coeffs = np.array([0.05, 0.02, 4, 0.03])

boids = np.zeros((N, 6), dtype=np.float64) # x, y, vx, vy, ax, ay

In [31]:
def init_boids(boids: np.ndarray, asp: float, vrange: tuple = (0., 1.)):
    n = boids.shape[0]
    rng = np.random.default_rng()
    boids[:, 0] = rng.uniform(0., asp, size=n)
    boids[:, 1] = rng.uniform(0., 1., size=n)
    alpha = rng.uniform(0., 2 * np.pi, size=n)
    v = rng.uniform(*vrange, size=n)
    boids[:, 2] = v * np.cos(alpha)
    boids[:, 3] = v * np.sin(alpha)


def directions(boids: np.ndarray, dt: float):
    """

    Parameters
    ----------
    boids
    dt

    Returns
    -------
    array of N x (x0, y0, x1, y1) for array painting
    """
    return np.hstack((
        boids[:, :2] - dt * boids[:, 2:4],
        boids[:, :2]
    ))


In [32]:
init_boids(boids, asp)

In [33]:
canvas = scene.SceneCanvas(show=True, size=(w, h))
view = canvas.central_widget.add_view()
view.camera = scene.PanZoomCamera(rect=Rect(0, 0, asp, 1))
arrows = scene.Arrow(arrows=directions(boids, dt),
                     arrow_color=(1, 1, 1, 1),
                     arrow_size=5,
                     connect='segments',
                     parent=view.scene)

In [34]:
def update(event):
    arrows.set_data(arrows=directions(boids, dt))
    canvas.update()

In [35]:
if __name__ == '__main__':
    timer = app.Timer(interval=0, start=True, connect=update)
    canvas.measure_fps()
    app.run()